Inicialización de paquetes para Python

In [ ]:
from fbprophet import Prophet
import pandas as pd 
import matplotlib.pyplot as plt
import calendar

Modelo básico

In [ ]:
fecha = pd.Timestamp('2018-2-15')

datos = pd.read_table('Datos.csv',sep=';',decimal = ',', index_col=0)
datos.index = pd.to_datetime(datos.index, format='%d/%m/%Y')
data = datos[datos.index <= fecha].copy()
data['ds'] = data.index
data = pd.DataFrame(data[['ds', 'TOTALES']])
data.columns = ['ds','y']

m = Prophet()
m.fit(data)

future = m.make_future_dataframe(periods=365)

forecast = m.predict(future)
forecast.loc[forecast['yhat']<0, 'yhat'] = 0
forecast.loc[forecast['yhat_lower']<0, 'yhat_lower'] = 0
forecast.loc[forecast['yhat_upper']<0, 'yhat_upper'] = 0

fig1 = m.plot(forecast)
fig2 = m.plot_components(forecast)

Modelo básico + dias festivos

In [ ]:
fecha = pd.Timestamp('2018-2-15')

datos = pd.read_table('Datos.csv',sep=';',decimal = ',', index_col=0)
datos.index = pd.to_datetime(datos.index, format='%d/%m/%Y')
data = datos[datos.index <= fecha].copy()
data['ds'] = data.index
data = pd.DataFrame(data[['ds', 'TOTALES']])
data.columns = ['ds','y']

festivos = pd.read_table('festivos.csv',sep=';',decimal = ',', encoding='latin-1')
festivos['ds'] = pd.to_datetime(festivos['ds'], format='%d/%m/%Y')

m = Prophet(holidays=festivos)
m.fit(data)

future = m.make_future_dataframe(periods=365)

forecast = m.predict(future)
forecast.loc[forecast['yhat']<0, 'yhat'] = 0
forecast.loc[forecast['yhat_lower']<0, 'yhat_lower'] = 0
forecast.loc[forecast['yhat_upper']<0, 'yhat_upper'] = 0

fig1 = m.plot(forecast)
fig2 = m.plot_components(forecast)

Modelo básico + días festivos + PIB

In [ ]:
fecha = pd.Timestamp('2018-2-15')

datos = pd.read_table('Datos.csv',sep=';',decimal = ',', index_col=0)
datos.index = pd.to_datetime(datos.index, format='%d/%m/%Y')
data = datos[datos.index <= fecha].copy()
data['ds'] = data.index
data = pd.DataFrame(data[['ds', 'TOTALES']])
data.columns = ['ds','y']

festivos = pd.read_table('festivos.csv',sep=';',decimal = ',', encoding='latin-1')
festivos['ds'] = pd.to_datetime(festivos['ds'], format='%d/%m/%Y')

PIB = pd.read_table('PIB.csv',sep=';',decimal = ',',index_col=0)
PIB.index = pd.to_datetime(PIB.index, format='%d/%m/%Y')
PIB_antes = PIB[PIB.index<=fecha].copy()
PIB_pred = pd.read_table('PIB prediccion.csv',sep=';',decimal = ',',index_col=0)
PIB_pred.index = pd.to_datetime(PIB_pred.index, format='%d/%m/%Y')
PIB_despues = PIB_pred[PIB_pred.index>fecha].copy()
economia_PIB = pd.concat([PIB_antes, PIB_despues])

data_with_regressors=data.join(economia_PIB)

m = Prophet(holidays=festivos)
m.add_regressor('anual', prior_scale=0.5, mode='multiplicative')
m.add_regressor('trimestral', prior_scale=0.5, mode='multiplicative')
m.fit(data_with_regressors)

future = economia_PIB.copy()
future.index.name = 'ds'
future = future.reset_index()

forecast = m.predict(future)
forecast.loc[forecast['yhat']<0, 'yhat'] = 0
forecast.loc[forecast['yhat_lower']<0, 'yhat_lower'] = 0
forecast.loc[forecast['yhat_upper']<0, 'yhat_upper'] = 0

fig1 = m.plot(forecast)
fig2 = m.plot_components(forecast)


Modelo básico + días festivos + PIB + un parámetro especial

In [ ]:
fecha = pd.Timestamp('2018-2-15')

datos = pd.read_table('Datos.csv',sep=';',decimal = ',', index_col=0)
datos.index = pd.to_datetime(datos.index, format='%d/%m/%Y')
data = datos[datos.index <= fecha].copy()
data['ds'] = data.index
data = pd.DataFrame(data[['ds', 'ASPIRADO']])
data.columns = ['ds','y']

festivos = pd.read_table('festivos.csv',sep=';',decimal = ',', encoding='latin-1')
festivos['ds'] = pd.to_datetime(festivos['ds'], format='%d/%m/%Y')

PIB = pd.read_table('PIB.csv',sep=';',decimal = ',',index_col=0)
PIB.index = pd.to_datetime(PIB.index, format='%d/%m/%Y')
PIB_antes = PIB[PIB.index<=fecha].copy()
PIB_pred = pd.read_table('PIB prediccion.csv',sep=';',decimal = ',',index_col=0)
PIB_pred.index = pd.to_datetime(PIB_pred.index, format='%d/%m/%Y')
PIB_despues = PIB_pred[PIB_pred.index>fecha].copy()
economia_PIB = pd.concat([PIB_antes, PIB_despues])

empleado = pd.DataFrame(datos[['EMPLEO']])

regressors=economia_PIB.join(empleado)
data_with_regressors=data.join(regressors)

m = Prophet(holidays=festivos)
m.add_regressor('anual', prior_scale=0.5, mode='multiplicative')
m.add_regressor('trimestral', prior_scale=0.5, mode='multiplicative')
m.add_regressor('EMPLEO', prior_scale=0.5, mode='multiplicative')
m.fit(data_with_regressors)

future = regressors.copy()
future.index.name = 'ds'
future = future.reset_index()
future['EMPLEO'] = future['EMPLEO'].fillna(2)

forecast = m.predict(future)
forecast.loc[forecast['yhat']<0, 'yhat'] = 0
forecast.loc[forecast['yhat_lower']<0, 'yhat_lower'] = 0
forecast.loc[forecast['yhat_upper']<0, 'yhat_upper'] = 0

fig1 = m.plot(forecast)
fig2 = m.plot_components(forecast)


Modelo básico + días festivos + PIB + Clima

In [ ]:
fecha = pd.Timestamp('2018-2-15')

datos = pd.read_table('Datos.csv',sep=';',decimal = ',', index_col=0)
datos.index = pd.to_datetime(datos.index, format='%d/%m/%Y')
data = datos[datos.index <= fecha].copy()
data['ds'] = data.index
data = pd.DataFrame(data[['ds', 'TOTALES']])
data.columns = ['ds','y']

festivos = pd.read_table('festivos.csv',sep=';',decimal = ',', encoding='latin-1')
festivos['ds'] = pd.to_datetime(festivos['ds'], format='%d/%m/%Y')

PIB = pd.read_table('PIB.csv',sep=';',decimal = ',',index_col=0)
PIB.index = pd.to_datetime(PIB.index, format='%d/%m/%Y')
PIB_antes = PIB[PIB.index<=fecha].copy()
PIB_pred = pd.read_table('PIB prediccion.csv',sep=';',decimal = ',',index_col=0)
PIB_pred.index = pd.to_datetime(PIB_pred.index, format='%d/%m/%Y')
PIB_despues = PIB_pred[PIB_pred.index>fecha].copy()
economia_PIB = pd.concat([PIB_antes, PIB_despues])

empleado = pd.DataFrame(datos[['EMPLEO']])

clima = pd.read_table('Clima.csv', sep=';', decimal = ',', index_col=0)
clima.index = pd.to_datetime(clima.index, format='%d/%m/%Y')
clima = pd.DataFrame(clima[['sol','tmin','tmax','prec']])
clima = clima.interpolate(method = 'linear')
clima_antes = clima[clima.index<=fecha].copy()
clima_pred = pd.read_table('Clima prediccion.csv', sep=';', decimal = ',', index_col=0)
clima_pred.index = pd.to_datetime(clima_pred.index, format='%d/%m/%Y')
clima_despues = clima_pred[clima_pred.index>fecha].copy()
climatologia = pd.concat([clima_antes, clima_despues])

regressors = economia_PIB.join(empleado)
regressors = regressors.join(climatologia)
data_with_regressors = data.join(regressors)

m = Prophet(holidays=festivos)
m.add_regressor('anual', prior_scale=0.5, mode='multiplicative')
m.add_regressor('trimestral', prior_scale=0.5, mode='multiplicative')
m.add_regressor('EMPLEO', prior_scale=0.5, mode='multiplicative')
m.add_regressor('sol', prior_scale=0.5, mode='multiplicative')
m.add_regressor('tmin', prior_scale=0.5, mode='multiplicative')
m.add_regressor('tmax', prior_scale=0.5, mode='multiplicative')
m.add_regressor('prec', prior_scale=0.5, mode='multiplicative')
m.fit(data_with_regressors)

future = regressors.copy()
future.index.name = 'ds'
future = future.reset_index()
future['EMPLEO'] = future['EMPLEO'].fillna(2)

forecast = m.predict(future)

forecast.loc[forecast['yhat']<0, 'yhat'] = 0
forecast.loc[forecast['yhat_lower']<0, 'yhat_lower'] = 0
forecast.loc[forecast['yhat_upper']<0, 'yhat_upper'] = 0

fig1 = m.plot(forecast)
fig2 = m.plot_components(forecast)

Función modelo_TFM (básico)

In [ ]:

def modelo_prophet(data,fecha, season, periodo, furier, modo):
    festivos = pd.read_table('festivos.csv',sep=';',decimal=',',encoding='latin-1')
    festivos['ds'] = pd.to_datetime(festivos['ds'], format = '%d/%m/%Y')
    
    PIB = pd.read_table('PIB.csv',sep = ';',decimal = ',',index_col=0)
    PIB.index = pd.to_datetime(PIB.index, format = '%d/%m/%Y')
    PIB_antes = PIB[PIB.index <= fecha].copy()
    PIB_pred = pd.read_table('PIB prediccion.csv',sep=';',decimal = ',',index_col=0)
    PIB_pred.index = pd.to_datetime(PIB_pred.index, format = '%d/%m/%Y')
    PIB_despues = PIB_pred[PIB_pred.index>fecha].copy()
    economia_PIB = pd.concat([PIB_antes, PIB_despues])
    
    empleado = pd.DataFrame(datos[['EMPLEO']])
    
    clima = pd.read_table('Clima.csv', sep = ';', decimal = ',', index_col=0)
    clima.index = pd.to_datetime(clima.index, format='%d/%m/%Y')
    clima = pd.DataFrame(clima[['sol','tmin','tmax','prec']])
    clima = clima.interpolate(method = 'linear')
    clima_antes = clima[clima.index <= fecha].copy()
    clima_pred = pd.read_table('Clima prediccion.csv',sep=';',decimal=',',index_col=0)
    clima_pred.index = pd.to_datetime(clima_pred.index, format = '%d/%m/%Y')
    clima_despues = clima_pred[clima_pred.index > fecha].copy()
    climatologia = pd.concat([clima_antes, clima_despues])
    
    regressors = economia_PIB.join(empleado)
    regressors = regressors.join(climatologia)
    data_with_regressors = data.join(regressors)
    
    m = Prophet(holidays = festivos, seasonality_mode = 'multiplicative', yearly_seasonality=20) #mcmc_samples=300)
    m.add_regressor('anual', prior_scale = 0.5, mode = 'multiplicative')
    m.add_regressor('trimestral', prior_scale = 0.5, mode = 'multiplicative')
    m.add_regressor('EMPLEO', prior_scale = 0.5, mode = 'multiplicative')
    m.add_regressor('sol', prior_scale = 0.5, mode = 'additive')
    m.add_regressor('tmin', prior_scale = 0.5, mode = 'additive')
    m.add_regressor('tmax', prior_scale = 0.5, mode = 'additive')
    m.add_regressor('prec', prior_scale = 0.5, mode = 'additive')
    if season: 
        for c in range(0,len(periodo)):
            m.add_seasonality('periodo'+str(c), period=periodo[c], fourier_order=furier[c], mode=modo[c])
    m.fit(data_with_regressors)
    
    future = regressors.copy()
    future.index.name = 'ds'
    future = future.reset_index()
    future['EMPLEO'] = future['EMPLEO'].fillna(2)
    forecast = m.predict(future)
    forecast.loc[forecast['yhat'] < 0, 'yhat'] = 0
    forecast.loc[forecast['yhat_lower'] < 0, 'yhat_lower'] = 0
    forecast.loc[forecast['yhat_upper'] < 0, 'yhat_upper'] = 0
    
    return m, forecast

def modelo_TFM(data,fecha, season, periodo, furier,modo):
    m, forecast = modelo_prophet(data,fecha, season, periodo, furier, modo)
    serie_anterior = data[data['ds'] <= fecha][['ds', 'y']]
    nueva_serie = forecast[forecast['ds'] > fecha][['ds', 'yhat']]
    nueva_serie.rename(columns = {'yhat':'y'}, inplace = True)
    serie = pd.concat((serie_anterior, nueva_serie))
    serie['FECHA'] = pd.to_datetime(serie['ds'], format = '%d/%m/%Y')
    serie.index = serie['FECHA']
    serie = pd.DataFrame(serie[['ds', 'y']])
    return m, forecast, round(serie[serie['ds'].dt.year == fecha.year]['y'].sum(),2)

Modelo inicial de referencia

In [ ]:
def modelo_inicial(data,fecha_estudio):
    año_estudio=fecha_estudio.year
    mes_estudio=fecha_estudio.month
    dia_estudio=fecha_estudio.day
    
#     display(data)
    datos_mes = data.groupby(pd.Grouper(freq='M')).sum()
#     display(datos_mes)
    datos_base = datos_mes.reset_index()
    datos_base = datos_base[(datos_base['FECHA'].dt.year < año_estudio)]
    datos_base['año'] = datos_base['FECHA'].dt.year
    datos_base['mes'] = datos_base['FECHA'].dt.month
#     display(datos_base)
    
    modelo_base = datos_base.groupby('mes').mean()['y']
    modelo_base = pd.DataFrame({'mes':modelo_base.index, 'y':modelo_base.values})
    modelo_base['peso'] = modelo_base.y.cumsum() / modelo_base.y.sum()
#     display(modelo_base)
    
    año_actual = data[(data['ds'].dt.year == año_estudio)]['y']
    año_actual = pd.DataFrame({'FECHA':año_actual.index, 'y':año_actual.values})
    mes_actual = año_actual[(año_actual['FECHA'].dt.month == mes_estudio)]
    acumulado_anterior = año_actual[(año_actual['FECHA'].dt.month < mes_estudio)]['y'].sum()
    acumulado_mes = mes_actual[(mes_actual['FECHA'].dt.day <= dia_estudio)]['y'].sum()
    prorrateo_mes = acumulado_mes * calendar.monthrange(año_estudio,mes_estudio)[1] / dia_estudio
    estimacion_año = (acumulado_anterior + prorrateo_mes) / modelo_base[modelo_base['mes'] == mes_estudio]
    return round(estimacion_año['peso'][mes_estudio-1], 2)

Script para generar DataSet de resultados comparativos

In [ ]:
datos = pd.read_table('Datos.csv',sep=';',decimal = ',', index_col = 0)
datos.index = pd.to_datetime(datos.index, format = '%d/%m/%Y')

datos_orig = datos.copy()
datos_orig['ds'] = datos_orig.index
datos_orig = pd.DataFrame(datos_orig[['ds', 'TOTALES']])
datos_orig.columns = ['ds','y']

data = datos.copy()
data['ds'] = data.index
data = pd.DataFrame(data[['ds', 'TOTALES']])
data.columns = ['ds','y']

comparison = []
comparison = pd.DataFrame(comparison, columns = ['ds' , 'real', 'base', 'prophet'])

for i in range(2010,2020):
    for j in range(1,13):
        for k in range(1,calendar.monthrange(i, j)[1]+1):
            fecha_estudio = pd.Timestamp(i, j, k)
            total_real = datos_orig[datos_orig['ds'].dt.year==i]['y'].sum()
            total_base = modelo_inicial(data,fecha_estudio)
            m, forecast, total_ph = modelo_TFM(data,fecha_estudio)
            print(fecha_estudio)
            comparison=comparison.append({'ds':fecha_estudio , 
                                          'real':total_real, 
                                          'base':total_base, 
                                          'prophet':total_ph},
                                         ignore_index=True)
            comparison.to_csv(r'D:\COSAS\Máster en Inteligencia Artificial\09. TFM\comparison_2.csv', 
                              index = False, sep = ';', decimal=',')

In [ ]:
comparison = pd.read_table('comparison.csv',sep=';',decimal = ',') #, index_col=0)
comparison['ds'] = pd.to_datetime(comparison['ds'], format = '%Y-%m-%d')
comparison=comparison[comparison['ds'].dt.year == 2019]
fig_aux, ax_aux = plt.subplots(facecolor='w', figsize=(15, 5)) 
ax_aux.plot(comparison['ds'],comparison['real'],label='real')
ax_aux.plot(comparison['ds'],comparison['base'],label='modelo inicial')
ax_aux.plot(comparison['ds'],comparison['prophet'],label='modelo Prophet')
ax_aux.legend(loc='best')
plt.show()

In [ ]:
año_estudio=2016
mes_estudio=1
dia_estudio=1
fecha_estudio=pd.Timestamp(año_estudio, mes_estudio, dia_estudio)

datos = pd.read_table('Datos.csv',sep=';',decimal = ',', index_col = 0)
datos.index = pd.to_datetime(datos.index, format = '%d/%m/%Y')

datos_orig = datos.copy()
datos_orig['ds'] = datos_orig.index
datos_orig = pd.DataFrame(datos_orig[['ds', 'TOTALES']])
datos_orig.columns = ['ds','y']

data = datos.copy()
data['ds'] = data.index
data = pd.DataFrame(data[['ds', 'TOTALES']])
data.columns = ['ds','y']

m, forecast, total_ph = modelo_TFM(data,fecha_estudio,True,91,6)

serie_calc = forecast  [forecast  ['ds'].dt.year == año_estudio].reset_index()[['ds', 'yhat']].copy()
serie_calc.rename(columns = {'yhat':'y_calc'}, inplace = True)
serie_real = datos_orig[datos_orig['ds'].dt.year == año_estudio].reset_index()[['ds', 'y']].copy()
serie_real.rename(columns = {'y':'y_real'}, inplace = True)
calculo = serie_calc.merge(serie_real)
calculo['diferencia'] = calculo['y_calc']-calculo['y_real']
calculo['acumulado'] = calculo['diferencia'].cumsum()
calculo_A2 = calculo.copy
fig_aux, ax_aux = plt.subplots(facecolor='w', figsize=(15, 5)) 
ax_aux.plot(calculo['ds'],calculo['acumulado'])

In [ ]:
def modelo_optim(periodo, furier, modo):
    m, forecast, total_ph = modelo_TFM(data,fecha_estudio,True,periodo,furier, modo)
    display(total_ph)
    serie_calc = forecast  [forecast  ['ds'].dt.year == año_estudio].reset_index()[['ds', 'yhat']].copy()
    serie_calc.rename(columns = {'yhat':'y_calc'}, inplace = True)
    serie_real = datos_orig[datos_orig['ds'].dt.year == año_estudio].reset_index()[['ds', 'y']].copy()
    serie_real.rename(columns = {'y':'y_real'}, inplace = True)
    calculo = serie_calc.merge(serie_real)
    calculo['diferencia'] = calculo['y_calc']-calculo['y_real']
    calculo['acumulado'] = calculo['diferencia'].cumsum()
    return calculo

In [ ]:
fecha_estudio = pd.Timestamp('2015-05-01')
modelo_optim([91,182],[5,2],['additive','additive'])

In [ ]:
# calculo_A1 = modelo_optim ([91],    [2]  ,['additive'])
# calculo_A2 = modelo_optim ([182],   [2] ,['additive'] )
# calculo_A3 = modelo_optim ([91],    [5] ,['multiplicative'] )
# calculo_A4 = modelo_optim ([182],   [5] ,['multiplicative'] )
# calculo_A5 = modelo_optim ([91,182],[5,2],['additive','additive'])
# calculo_A6 = modelo_optim ([91,182],[5,2],['multiplicative','additive'])
# calculo_A7 = modelo_optim ([91,182],[5,2],['additive','multiplicative'])
# calculo_A8 = modelo_optim ([91,182],[5,2],['multiplicative','multiplicative'])
# calculo_A9 = modelo_optim ([270],[2])
# calculo_A10 = modelo_optim([300],[2])
# calculo_A11 = modelo_optim([330],[2])
# calculo_A12 = modelo_optim([360],[2])
# calculo_A13 = modelo_optim([91,182],[5,2],['additive','additive'])
# calculo_A14 = modelo_optim([91,182],[5,2],['additive','additive'])
# calculo_A15 = modelo_optim([91,182],[5,2],['additive','additive'])
# calculo_A16 = modelo_optim([91,182],[5,2],['additive','additive'])
calculo_A17 = modelo_optim([91,182],[5,2],['additive','additive'])
fig_aux, ax_aux = plt.subplots(facecolor='w', figsize=(15, 5)) 
# ax_aux.plot(calculo_A1['ds'],calculo_A1['acumulado'],label='A1')
# ax_aux.plot(calculo_A2['ds'],calculo_A2['acumulado'],label='A2')
# ax_aux.plot(calculo_A3['ds'],calculo_A3['acumulado'],label='A3')
# ax_aux.plot(calculo_A4['ds'],calculo_A4['acumulado'],label='A4')
ax_aux.plot(calculo_A5['ds'],calculo_A5['acumulado'],label='Con estacionalidad 3, 6 y 12 meses')
# ax_aux.plot(calculo_A6['ds'],calculo_A6['acumulado'],label='A6')
# ax_aux.plot(calculo_A7['ds'],calculo_A7['acumulado'],label='A7')
# ax_aux.plot(calculo_A8['ds'],calculo_A8['acumulado'],label='A8')
# ax_aux.plot(calculo_A9['ds'],calculo_A9['acumulado'],label='A9')
# ax_aux.plot(calculo_A10['ds'],calculo_A10['acumulado'],label='A10')
# ax_aux.plot(calculo_A11['ds'],calculo_A11['acumulado'],label='A11')
# ax_aux.plot(calculo_A12['ds'],calculo_A12['acumulado'],label='A12')
ax_aux.plot(calculo_A13['ds'],calculo_A13['acumulado'],label='precipitaciones "additive"')
ax_aux.plot(calculo_A14['ds'],calculo_A14['acumulado'],label='todo clima "additive"')
ax_aux.plot(calculo_A15['ds'],calculo_A15['acumulado'],label='estacionalidad "multiplicativa"')
ax_aux.plot(calculo_A16['ds'],calculo_A16['acumulado'],label='muestreo bayesiano')
ax_aux.plot(calculo_A17['ds'],calculo_A17['acumulado'],label='estacionalidad anual ampliada')
ax_aux.legend(loc='best')
# plt.legend(loc="upper left")
plt.show()
# seasonality_mode = 'multiplicative', yearly_seasonality=20) #mcmc_samples=300

In [ ]:
fig_aux, ax_aux = plt.subplots(facecolor='w', figsize=(15, 5)) 
ax_aux.plot(calculo_A1['ds'],calculo_A1['acumulado'])

In [ ]:
# comparison=[]
comparison_2 = pd.read_table('comparison_2.csv',sep=';',decimal = ',') #, index_col=0)
comparison_2['ds'] = pd.to_datetime(comparison_2['ds'], format = '%d/%m/%Y')
comparison_3=comparison_2[comparison_2['ds'].dt.year == 2015]
# comparison_3=comparison_3[comparison_3['ds'].dt.month == 8]
# comparison_3=comparison_2[comparison_2['ds'].str[:4]=='2016']
fig_aux, ax_aux = plt.subplots(facecolor='w', figsize=(15, 5)) 
ax_aux.plot(comparison_3['ds'],comparison_3['real'],label='real')
ax_aux.plot(comparison_3['ds'],comparison_3['base'],label='modelo inicial')
ax_aux.plot(comparison_3['ds'],comparison_3['prophet'],label='modelo Prophet')
ax_aux.legend(loc='best')
plt.show()

In [ ]:
fecha = pd.Timestamp('2018-2-15')
datos = pd.read_table('Datos.csv',sep=';',decimal = ',', index_col = 0)
datos.index = pd.to_datetime(datos.index, format = '%d/%m/%Y')
data = datos[datos.index <= fecha].copy()
data['ds'] = data.index
data = pd.DataFrame(data[['ds', 'TOTALES']])
data.columns = ['ds','y']

m, forecast, total_ph = modelo_TFM(data,fecha)
base = modelo_inicial(data,fecha)
real = round(data[data['ds'].dt.year == fecha.year]['y'].sum(),2)

print("{valor:.2f}".format(valor = total_ph))
print("{valor:.2f}".format(valor = base))
print("{valor:.2f}".format(valor = real))


In [ ]:
año_estudio=2018
mes_estudio=1
dia_estudio=1
fecha_estudio=pd.Timestamp(año_estudio, mes_estudio, dia_estudio)

ingresos_pred=forecast[forecast['ds']>fecha_estudio][['ds', 'yhat']].copy()
ingresos_pred.rename(columns = {'yhat':'y'}, inplace = True)
ingresos_real=datos_orig[datos_orig['ds']<=fecha_estudio][['ds', 'y']]
ingresos = pd.concat((ingresos_real, ingresos_pred))
ingresos['FECHA'] = pd.to_datetime(ingresos['ds'], format='%d/%m/%Y')
ingresos.index = ingresos['FECHA'] 
ingresos = pd.DataFrame(ingresos[['ds', 'y']])

calculo=ingresos[ingresos['ds'].dt.year==año_estudio].copy()
calculo['diferencia'] = ingresos[ingresos['ds'].dt.year==año_estudio]['y']-datos_orig[datos_orig['ds'].dt.year==año_estudio]['y']
calculo['acumulado'] = calculo['diferencia'].cumsum()

calculo_2 = ingresos[ingresos['ds'].dt.year==año_estudio].copy()
calculo_2['acumulado'] = ingresos['y'].cumsum()

calculo_3 = datos_orig[datos_orig['ds'].dt.year==año_estudio].copy()
calculo_3['acumulado'] = datos_orig['y'].cumsum()


fig_aux, ax_aux = plt.subplots(facecolor='w', figsize=(15, 5)) 
# ax_aux.plot(calculo['ds'],calculo['diferencia'])
ax_aux.plot(calculo['ds'],calculo['acumulado'])
# ax_aux.plot(calculo_2['ds'],calculo_2['acumulado'])
# ax_aux.plot(calculo_3['ds'],calculo_3['acumulado'])
# display(calculo_2)

In [ ]:
fig_aux, ax_aux = plt.subplots(facecolor='w', figsize=(15, 5)) 
cv = df_cv[df_cv['cutoff'] == pd.Timestamp('2016-08-19')]
# cv.loc[cv['yhat']<0, 'yhat'] = 0
# cv.loc[cv['yhat_lower']<0, 'yhat_lower'] = 0
# cv.loc[cv['yhat_upper']<0, 'yhat_upper'] = 0
ax_aux.plot(cv['ds'],cv['yhat'])
ax_aux.plot(cv['ds'],cv['y'])
fig1 = m.plot(cv)

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(cv, metric='mse')

In [ ]:
from fbprophet.diagnostics import performance_metrics
# performance_metrics?
df_p = performance_metrics(cv, rolling_window=0.9)
df_p.head()